In [1]:
import os
from PIL import Image
import numpy as np

def convert_png_to_lvgl(png_path, output_dir):
    # Load the PNG image
    img = Image.open(png_path)
    
    # Convert to RGBA if not already
    if img.mode != 'RGBA':
        img = img.convert('RGBA')
        
    # Create a new image with white color while preserving alpha
    img_data = np.array(img)
    alpha = img_data[:, :, 3]
    white_img = np.zeros_like(img_data)
    white_img[:, :, 0] = 255  # Red channel
    white_img[:, :, 1] = 255  # Green channel
    white_img[:, :, 2] = 255  # Blue channel
    white_img[:, :, 3] = alpha  # Preserve original alpha
    
    # Convert back to PIL Image
    white_img = Image.fromarray(white_img)
    
    # Resize to 40x40 to ensure proper centering in 60x60 squares
    white_img = white_img.resize((40, 40), Image.Resampling.LANCZOS)
    
    # Create a new 60x60 transparent image
    final_img = Image.new('RGBA', (60, 60), (0, 0, 0, 0))
    
    # Calculate position to paste (center)
    paste_x = (60 - 40) // 2
    paste_y = (60 - 40) // 2
    
    # Paste the resized white image in the center
    final_img.paste(white_img, (paste_x, paste_y), white_img)
    
    # Convert to numpy array for LVGL format
    img_data = np.array(final_img)
    
    # Get base filename without extension
    base_name = os.path.splitext(os.path.basename(png_path))[0]
    
    # Create output C file path
    c_file_path = os.path.join(output_dir, f'{base_name}.c')
    
    with open(c_file_path, 'w') as f:
        # Write header
        f.write('#include "lvgl.h"\n\n')
        
        # Write image data array
        f.write('const LV_ATTRIBUTE_MEM_ALIGN LV_ATTRIBUTE_LARGE_CONST uint8_t {}_map[] = {{\n'.format(base_name))
        
        # Convert pixel data to LVGL format
        pixels = []
        for y in range(img_data.shape[0]):
            for x in range(img_data.shape[1]):
                r, g, b, a = img_data[y, x]
                pixels.append(a)  # Alpha channel
        
        # Write bytes in hex format, 16 per line
        for i in range(0, len(pixels), 16):
            line = pixels[i:i+16]
            hex_values = [f'0x{x:02x}' for x in line]
            f.write('    ' + ', '.join(hex_values) + ',\n')
        
        f.write('};\n\n')
        
        # Write image descriptor
        f.write(f'const lv_img_dsc_t {base_name} = {{\n')
        f.write('    .header.always_zero = 0,\n')
        f.write('    .header.w = 60,\n')
        f.write('    .header.h = 60,\n')
        f.write('    .data_size = 3600,\n')
        f.write('    .header.cf = LV_IMG_CF_TRUE_COLOR_ALPHA,\n')
        f.write(f'    .data = {base_name}_map,\n')
        f.write('};\n')

# List of PNG files to convert
png_files = [
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\bluetooth.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\folder.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\github.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\power.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\radio.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\server.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\settings.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\sun.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\tv.png',
    r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\png\wifi.png'
]

# Output directory for C files
output_dir = r'C:\Users\Giovane\Documents\PlatformIO\Projects\S3-Factory-Clock - Copy\src\icons'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Convert each PNG file
for png_file in png_files:
    convert_png_to_lvgl(png_file, output_dir) 